In [ ]:
import serial
import time

def send_cmd(com_port, string1, display):
    ser = serial.Serial(port=com_port, baudrate=115200, parity=serial.PARITY_NONE,
                        stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=0.1)
    to_send = string1.encode() + b'\r\n'
    ser.write(to_send)
    time.sleep(0.5)  # Delay for response
    response = ser.read(ser.in_waiting or 1).decode('utf-8')
    num = len(response)
    if num == 0:
        print('Timeout was reached.')
    elif display:
        print(f'String sent to Device: "{string1}"')
        print(f'Response from Device (in HEX): "{response.strip()}"')
        print(f'Number of Chars received: {num - 2} (+2 terminator chars)')
    ser.close()
    return response, num

def add_error_code(string):
    if len(string) % 2 == 0:
        print('Error: Input string length must be odd.')
        return -1
    temp = 0x100000000  # Equivalent to hex2dec('ffffffff') + 1
    string = string[1:]  # Remove the first character
    for i in range(0, len(string) - 1, 2):
        temp -= int(string[i:i+2], 16)
    temp_hex = '{:X}'.format(temp)
    complete_string = ':' + string + temp_hex[-2:]
    return complete_string

def home_x(com_port):
    # Switch Modbus 'x' on
    command = add_error_code(':03050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':03050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0305040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0305040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':030390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F0" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def get_position_x(com_port):
    # Send the command to get the X position
    command = add_error_code(':030390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position

def format_hex(value, length):
    """Format the given value as a hexadecimal string of the specified length."""
    return f'{value:0{length}X}'

def calculate_checksum(command):
    """Calculate the two's complement checksum for the given command."""
    sum_bytes = sum(int(command[i:i+2], 16) for i in range(1, len(command), 2))  # Start from 1 to skip ':'
    checksum = (0x10000 - sum_bytes) & 0xFFFF
    return format_hex(checksum, 4)[-2:]


def move_x(com_port, distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'03109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    # Print the constructed command for debugging
    print(f"Constructed command for sending: {complete_command_with_checksum}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum, True)


def home_y(com_port):
    # Switch Modbus 'x' on
    command = add_error_code(':02050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':02050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0205040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0205040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':020390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F1" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1


def get_position_y(com_port):
    # Send the command to get the X position
    command = add_error_code(':020390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position


def move_y(com_port, distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'02109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    # Print the constructed command for debugging
    print(f"Constructed command for sending: {complete_command_with_checksum}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum, True)



def home_z(com_port):
    # Switch Modbus 'z' on
    command = add_error_code(':01050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on
    command = add_error_code(':01050403FF00')
    send_cmd(com_port, command, True)

    # Home Device
    command = add_error_code(':0105040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command
    command = add_error_code(':0105040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':010390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F2" in response:
            print("Homing process completed successfully.")
            homing_completed = True
        else:
            print("Waiting for homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1


def get_position_z(com_port):
    # Send the command to get the Z position
    command = add_error_code(':010390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    # Adjusted index for Python string and multiplied by 2 since each hex digit represents half a byte
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    return position

def move_z(distance, speed, acceleration):
    """Construct the move Z command based on distance, speed, and acceleration."""
    if distance < 0 or distance > 150 or speed <= 0 or speed > 100:
        print("ERROR: Invalid Position or Speed.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'01109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    print(f"Constructed command for sending: {complete_command_with_checksum}")
    return complete_command_with_checksum


def home_phi(com_port):
    # Switch Modbus 'x' on
    command = add_error_code(':04050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':04050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0405040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0405040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':040390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98EF" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def get_position_phi(com_port):
    # Send the command to get the X position
    command = add_error_code(':040390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position


def move_phi(com_port, distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'04109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    # Print the constructed command for debugging
    print(f"Constructed command for sending: {complete_command_with_checksum}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum, True)


# Main sequence
def main():
    com_port = 'COM3'  # Adjust to your COM port
    distance = 40  # Desired distance in mm
    speed = 25  # Desired speed in mm/s
    acceleration = 0.3  # Desired acceleration in mm/s^2
    timeout = 0.5  # Timeout for waiting after command sent

    # Home the X stage first
    print("Homing X stage...")
    home_x(com_port)  # Ensure home_x function is defined and working as expected

    # Generate the move X command
    print(f"Moving X stage to {distance}mm at {speed}mm/s with {acceleration}mm/s^2 acceleration.")
    move_x(com_port, distance, speed, acceleration)  # Ensure move_x is correctly implemented

    # Wait for movement to complete or implement additional checks as needed
    #time.sleep(timeout)  # Simple wait; adjust based on your system's needs

    # Retrieve and print the final position of the X stage
    final_position_x = get_position_x(com_port)  # Ensure get_position_x is correctly implemented
    print(f"Final X position: {final_position_x}mm")

    # Home the Y stage first
    print("Homing X stage...")
    home_y(com_port)  # Ensure home_x function is defined and working as expected

    # Generate the move Y command
    print(f"Moving X stage to {distance}mm at {speed}mm/s with {acceleration}mm/s^2 acceleration.")
    move_y(com_port, distance, speed, acceleration)  # Ensure move_x is correctly implemented

    # Wait for movement to complete or implement additional checks as needed
    #time.sleep(timeout)  # Simple wait; adjust based on your system's needs

    # Retrieve and print the final position of the Y stage
    final_position_x = get_position_x(com_port)  # Ensure get_position_x is correctly implemented
    print(f"Final X position: {final_position_x}mm")
    
    # Home the Z stage first
    print("Homing Z stage...")
    home_z(com_port)  # Ensure home_z function is defined and working as expected

    # Generate the move Z command
    move_command = move_z(distance, speed, acceleration)
    if move_command:
        # Send the move Z command
        print(f"Moving Z stage to {distance}mm at {speed}mm/s with {acceleration}mm/s^2 acceleration.")
        send_cmd(com_port, move_command, True)

        # Wait for movement to complete or implement additional checks as needed
        time.sleep(timeout)  # Simple wait; adjust based on your system's needs

        # Optionally, retrieve and print the final position
        final_position = get_position_z(com_port)  # Ensure get_position_z is correctly implemented
        print(f"Final Z position: {final_position}mm")

    # Home the Phi stage first
    print("Homing Phi stage...")
    home_phi(com_port)  # Ensure home_x function is defined and working as expected

    # Generate the move Phi command
    print(f"Moving Phi stage to {distance}mm at {speed}mm/s with {acceleration}mm/s^2 acceleration.")
    move_phi(com_port, distance, speed, acceleration)  # Ensure move_x is correctly implemented

    # Wait for movement to complete or implement additional checks as needed
    #time.sleep(timeout)  # Simple wait; adjust based on your system's needs

    # Retrieve and print the final position of the Phi stage
    final_position_x = get_position_phi(com_port)  # Ensure get_position_x is correctly implemented
    print(f"Final Phi position: {final_position_x}mm")


if __name__ == "__main__":
    main()

# Initialize Gamepad

In [ ]:
import pygame
import time

# Initialize Pygame and the joystick module
pygame.init()
pygame.joystick.init()

# Setup
joystick_detected = False
if pygame.joystick.get_count() > 0:
    joystick = pygame.joystick.Joystick(0)  # Get the first joystick
    joystick.init()
    print(f"Joystick initialized: {joystick.get_name()}")
    joystick_detected = True
else:
    print("No joystick detected")

def process_joystick_input():
    # Read analog sticks: Axis 0 and 1 for the left stick, 2 and 3 for the right stick
    left_stick_x = joystick.get_axis(0)
    left_stick_y = joystick.get_axis(1)
    right_stick_x = joystick.get_axis(2)
    right_stick_y = joystick.get_axis(3)

    # For demonstration, print out the values
    print(f"Left Stick X: {left_stick_x:.2f}, Y: {left_stick_y:.2f}")
    print(f"Right Stick X: {right_stick_x:.2f}, Y: {right_stick_y:.2f}")

    # Here, add your code to map these joystick positions to stage movements

# Main loop
try:
    while joystick_detected:
        pygame.event.pump()  # Process event queue
        process_joystick_input()
        time.sleep(0.1)  # Adding a small delay to make the output readable
except KeyboardInterrupt:
    print("Program exited")

pygame.quit()


In [2]:
import pygame
import serial
import time

com_port = 'COM3'  # Adjust to your COM port
distance = 10  # Desired distance in mm for X axis
speed = 15  # Desired speed in mm/s for X axis
acceleration = 0.5  # Desired acceleration in mm/s^2 for X axis


def send_cmd(com_port, string1, display):
    ser = serial.Serial(port=com_port, baudrate=115200, parity=serial.PARITY_NONE,
                        stopbits=serial.STOPBITS_ONE, bytesize=serial.EIGHTBITS, timeout=0.1)
    to_send = string1.encode() + b'\r\n'
    ser.write(to_send)
    time.sleep(0.5)  # Delay for response
    response = ser.read(ser.in_waiting or 1).decode('utf-8')
    num = len(response)
    if num == 0:
        print('Timeout was reached.')
    elif display:
        print(f'String sent to Device: "{string1}"')
        print(f'Response from Device (in HEX): "{response.strip()}"')
        print(f'Number of Chars received: {num - 2} (+2 terminator chars)')
    ser.close()
    return response, num

def add_error_code(string):
    if len(string) % 2 == 0:
        print('Error: Input string length must be odd.')
        return -1
    temp = 0x100000000  # Equivalent to hex2dec('ffffffff') + 1
    string = string[1:]  # Remove the first character
    for i in range(0, len(string) - 1, 2):
        temp -= int(string[i:i+2], 16)
    temp_hex = '{:X}'.format(temp)
    complete_string = ':' + string + temp_hex[-2:]
    return complete_string


def home_x(com_port):
    # Switch Modbus 'x' on
    command = add_error_code(':03050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':03050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0305040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0305040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':030390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F0" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def home_y(com_port):
    # Switch Modbus 'x' on
    command = add_error_code(':02050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':02050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0205040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0205040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':020390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F1" in response:
            print("Homing process for X axis completed successfully.")
            homing_completed = True
        else:
            print("Waiting for X axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1


def home_z(com_port):
    # Switch Modbus 'z' on
    command = add_error_code(':01050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on
    command = add_error_code(':01050403FF00')
    send_cmd(com_port, command, True)

    # Home Device
    command = add_error_code(':0105040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command
    command = add_error_code(':0105040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':010390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98F2" in response:
            print("Homing process completed successfully.")
            homing_completed = True
        else:
            print("Waiting for homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def home_phi(com_port):
    # Switch Modbus 'x' on
    command = add_error_code(':04050427FF00')
    send_cmd(com_port, command, True)

    # Switch SERVO on for X axis
    command = add_error_code(':04050403FF00')
    send_cmd(com_port, command, True)

    # Home X axis
    command = add_error_code(':0405040BFF00')
    send_cmd(com_port, command, True)

    # Homing off command for X axis
    command = add_error_code(':0405040B0000')
    send_cmd(com_port, command, True)

    # Flag to indicate completion
    homing_completed = False

    while not homing_completed:
        command = add_error_code(':040390050001')
        response, _ = send_cmd(com_port, command, True)
        
        if "98EF" in response:
            print("Homing process for Phi axis completed successfully.")
            homing_completed = True
        else:
            print("Waiting for Phi axis homing to complete... Response: ", response)

        time.sleep(0.5)

    return 1

def get_position_phi(com_port):
    # Send the command to get the X position
    command = add_error_code(':040390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position

def get_position_z(com_port):
    # Send the command to get the Z position
    command = add_error_code(':010390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    # Adjusted index for Python string and multiplied by 2 since each hex digit represents half a byte
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    return position

def get_position_y(com_port):
    # Send the command to get the X position
    command = add_error_code(':020390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position

def get_position_x(com_port):
    # Send the command to get the X position
    command = add_error_code(':030390000002')
    response, _ = send_cmd(com_port, command, True)
    
    # Extract the number of bytes encoding the position from the response
    n_bytes = int(response[5:7], 16)  # Adjusted index for Python string
    
    # Extract the position encoded in the response
    position_hex = response[7:7 + 2*n_bytes]
    position = int(position_hex, 16) / 100.0  # Convert from 1:100 mm to mm
    
    print(f"Current X position: {position}mm")
    return position

def format_hex(value, length):
    """Format the given value as a hexadecimal string of the specified length."""
    return f'{value:0{length}X}'

def calculate_checksum(command):
    """Calculate the two's complement checksum for the given command."""
    sum_bytes = sum(int(command[i:i+2], 16) for i in range(1, len(command), 2))  # Start from 1 to skip ':'
    checksum = (0x10000 - sum_bytes) & 0xFFFF
    return format_hex(checksum, 4)[-2:]


def move_x(com_port, distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'03109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    # Print the constructed command for debugging
    print(f"Constructed command for sending: {complete_command_with_checksum}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum, True)

def move_y(com_port, distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for X axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'02109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    # Print the constructed command for debugging
    print(f"Constructed command for sending: {complete_command_with_checksum}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum, True)

def move_z(com_port, distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for Z axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'01109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    # Print the constructed command for debugging
    print(f"Constructed command for sending: {complete_command_with_checksum}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum, True)

def move_phi(com_port, distance, speed, acceleration):
    if not (0 <= distance <= 50) or not (0 < speed <= 100) or not (0 <= acceleration):
        print("ERROR: Invalid distance, speed, or acceleration values for Phi axis.")
        return

    # Convert distance, speed, and acceleration to hexadecimal
    distance_hex = format_hex(int(distance * 100), 8)
    speed_hex = format_hex(int(speed * 100), 8)
    acceleration_hex = format_hex(int(acceleration * 100), 4)

    # Correct placement of speed and acceleration in the command string
    command = f'04109900000912{distance_hex}00000001{speed_hex[:4]}{speed_hex[4:]}{acceleration_hex}00000000'

    # Calculate and append the checksum
    complete_command = f':{command}'
    checksum = calculate_checksum(complete_command)
    complete_command_with_checksum = f'{complete_command}{checksum}'

    # Print the constructed command for debugging
    print(f"Constructed command for sending: {complete_command_with_checksum}")

    # Send the constructed command to the device
    send_cmd(com_port, complete_command_with_checksum, True)



def initialize_gamepad():
    pygame.init()
    pygame.joystick.init()
    joystick_count = pygame.joystick.get_count()
    if joystick_count == 0:
        print("No gamepad connected")
        return None
    else:
        joystick = pygame.joystick.Joystick(0)
        joystick.init()
        print(f"Gamepad connected: {joystick.get_name()}")
        return joystick

def poll_gamepad(joystick, com_port):
    # Updated button mappings to include phi controls
    buttons = {
        'home_x': 7,
        'move_x_increase': 14,
        'move_x_decrease': 13,
        'home_y': 8,
        'move_y_increase': 12,
        'move_y_decrease': 11,
        'home_z': 2,
        'move_z_increase': 0,
        'move_z_decrease': 3,
        'home_phi': 1,        # Homing phi axis
        'move_phi_increase': 9,  # Increase phi stage
        'move_phi_decrease': 10, # Decrease phi stage
        'exit': 15
    }

    # Assuming step_size and initial distances for X, Y, Z, and phi are defined
    acceleration = 0.5
    distance_x = 0
    distance_y = 0
    distance_z = 0  # Initialize Z distance
    step_size = 5
    distance_phi = 0  # Initialize phi distance
    step_size_phi = 5  # Define step size for phi, adjust as needed

    running = True
    while running:
        pygame.event.pump()
        # Include checks and actions for X, Y, Z stages as before

        if joystick.get_button(buttons['home_x']):
            print("Homing X axis...")
            home_x(com_port)

        if joystick.get_button(buttons['move_x_increase']):
            distance_x += step_size
            print(f"Increasing distance X: Moving X stage to {distance_x}mm")
            move_x(com_port, distance_x, step_size, 0.5)

        if joystick.get_button(buttons['move_x_decrease']):
            distance_x -= step_size
            distance_x = max(0, distance_x)
            print(f"Decreasing distance X: Moving X stage to {distance_x}mm")
            move_x(com_port, distance_x, step_size, 0.5)

        if joystick.get_button(buttons['home_y']):
            print("Homing Y axis...")
            home_y(com_port)

        if joystick.get_button(buttons['move_y_increase']):
            distance_y += step_size
            print(f"Increasing distance Y: Moving Y stage to {distance_y}mm")
            move_y(com_port, distance_y, step_size, 0.5)

        if joystick.get_button(buttons['move_y_decrease']):
            distance_y -= step_size
            distance_y = max(0, distance_y)
            print(f"Decreasing distance Y: Moving Y stage to {distance_y}mm")
            move_y(com_port, distance_y, step_size, 0.5)

        # Z stage control
        if joystick.get_button(buttons['home_z']):
            print("Homing Z axis...")
            home_z(com_port)

        if joystick.get_button(buttons['move_z_increase']):
            distance_z += step_size
            print(f"Increasing distance Z: Moving Z stage to {distance_z}mm")
            move_z(com_port, distance_z, step_size, 0.5)

        if joystick.get_button(buttons['move_z_decrease']):
            distance_z -= step_size
            distance_z = max(0, distance_z)
            print(f"Decreasing distance Z: Moving Z stage to {distance_z}mm")
            move_z(com_port, distance_z, step_size, 0.5)

        # Phi stage control
        if joystick.get_button(buttons['home_phi']):
            home_phi(com_port)

        if joystick.get_button(buttons['move_phi_increase']):
            distance_phi += step_size_phi
            move_phi(com_port, distance_phi, step_size_phi, acceleration)  # Increase phi stage

        if joystick.get_button(buttons['move_phi_decrease']):
            distance_phi -= step_size_phi
            move_phi(com_port, distance_phi, step_size_phi, acceleration)  # Decrease phi stage

        if joystick.get_button(buttons['exit']):
            print("Exiting program...")
            running = False

        time.sleep(0.1)

def main():
    com_port = 'COM3'  # Adjust to your COM port

    joystick = initialize_gamepad()
    if joystick is not None:
        poll_gamepad(joystick, com_port)
    pygame.quit()

if __name__ == "__main__":
    main()

Gamepad connected: PS4 Controller
Increasing distance X: Moving X stage to 5mm
Constructed command for sending: :03109900000912000001F400000001000001F40032000000001C
String sent to Device: ":03109900000912000001F400000001000001F40032000000001C"
Response from Device (in HEX): ":0310990000094B"
Number of Chars received: 15 (+2 terminator chars)
Increasing distance X: Moving X stage to 10mm
Constructed command for sending: :03109900000912000003E800000001000001F400320000000026
String sent to Device: ":03109900000912000003E800000001000001F400320000000026"
Response from Device (in HEX): ":0310990000094B"
Number of Chars received: 15 (+2 terminator chars)
Increasing distance X: Moving X stage to 15mm
Constructed command for sending: :03109900000912000005DC00000001000001F400320000000030
String sent to Device: ":03109900000912000005DC00000001000001F400320000000030"
Response from Device (in HEX): ":0310990000094B"
Number of Chars received: 15 (+2 terminator chars)
Increasing distance X: Moving X

# Check Gamepad Mapping

In [20]:
import pygame

def initialize_pygame():
    pygame.init()
    pygame.joystick.init()
    print(f"Detected {pygame.joystick.get_count()} joystick(s)")

def check_controllers():
    joystick_count = pygame.joystick.get_count()
    for i in range(joystick_count):
        joystick = pygame.joystick.Joystick(i)
        joystick.init()
        print(f"Joystick {i}: {joystick.get_name()}. It has {joystick.get_numbuttons()} buttons, {joystick.get_numaxes()} axes, and {joystick.get_numhats()} hats.")

def monitor_joystick(joystick, threshold=0.1):
    print("Monitoring joystick: Press buttons or move axes. Press Button 10 (usually L3 on PS4) to exit.")
    running = True
    while running:
        for event in pygame.event.get():  # User did something
            if event.type == pygame.JOYBUTTONDOWN:
                print(f"Button {event.button} pressed.")
                if event.button == 10:  # Adjust as necessary for your exit condition
                    running = False
            elif event.type == pygame.JOYBUTTONUP:
                print(f"Button {event.button} released.")
            elif event.type == pygame.JOYAXISMOTION:
                if abs(event.value) > threshold:  # Only print if movement exceeds threshold
                    print(f"Axis {event.axis} moved to {event.value}")
            elif event.type == pygame.JOYHATMOTION:
                print(f"Hat {event.hat} moved to {event.value}")

if __name__ == "__main__":
    initialize_pygame()
    if pygame.joystick.get_count() > 0:
        check_controllers()
        joystick = pygame.joystick.Joystick(0)
        joystick.init()
        monitor_joystick(joystick)
    else:
        print("No joystick detected.")


Detected 1 joystick(s)
Joystick 0: PS4 Controller. It has 16 buttons, 6 axes, and 0 hats.
Monitoring joystick: Press buttons or move axes. Press Button 10 (usually L3 on PS4) to exit.
Button 3 pressed.
Button 3 released.
Button 0 pressed.
Button 0 released.


KeyboardInterrupt: 